Explorando dados do Starbucks

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import seaborn as sns
import os
import sys
import importlib
import functions as aux_fun
importlib.reload(aux_fun)
from datetime import datetime as dti

Lendo os dados

In [2]:
# Lendo os dados

portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

Dicionário dos dados:  

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [3]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [4]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [5]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [6]:
portfolio.shape, profile.shape, transcript.shape

((10, 6), (17000, 5), (306534, 4))

Mapping to integer   

O código hexadecimal é grande para trabalhar

In [8]:

map_portifolio = json.load(open('mapper_id/portifolio_ids.json'))
map_profile = json.load(open('mapper_id/profile_ids.json'))


Olhada na timeline

In [9]:
# Por usuário
cond = transcript.person == '8978c47b823e465a82a576a32680585f'
transcript.loc[cond]

,person,event,value,time
873,8978c47b823e465a82a576a32680585f,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0
25422,8978c47b823e465a82a576a32680585f,transaction,{'amount': 28.91},30
28908,8978c47b823e465a82a576a32680585f,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},42
54032,8978c47b823e465a82a576a32680585f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},168
105555,8978c47b823e465a82a576a32680585f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},300
112574,8978c47b823e465a82a576a32680585f,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336
125831,8978c47b823e465a82a576a32680585f,offer viewed,{'offer id': '3f207df678b143eea3cee63160fa8bed'},336
125832,8978c47b823e465a82a576a32680585f,transaction,{'amount': 8.89},336
136611,8978c47b823e465a82a576a32680585f,transaction,{'amount': 15.0},360
158565,8978c47b823e465a82a576a32680585f,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},408


Todos os usuários receberam ofertas?

In [10]:
transcript.loc[transcript.event=='offer received'].person.nunique() - profile.id.nunique() # .groupby('person', dropna=False).agg(count=('person','count')).sort_values('count')

# 6 usuários não receberam oferta

-6

Explorando a coluna value

In [11]:
# Abrindo a coluna de value

# offer_id é para ofertas completas e offer id é para ofertass enviadas

df_temp = pd.concat([transcript,pd.DataFrame(transcript.value.to_list())], axis=1)

df_temp.loc[transcript.event=='offer received']. \
    groupby(['person', 'offer id']).aggregate('count').iloc[100:110]



event  \
person                           offer id                                  
00aee28bbb3848dd8a31f0c91dc267dd ae264e3637204a6fb9bb56bc8210ddfd      1   
                                 f19421c1d4aa40978ebb69ca19b0e20d      1   
00b18b535d6d4f779dea4dc9ac451478 2298d6c36e964ae4a3e7e9706d1fb8c2      2   
                                 5a8bc65990b245e5a138643cd4eb9837      1   
                                 ae264e3637204a6fb9bb56bc8210ddfd      1   
                                 fafdcd668e3743c1bb461111dcafc2a4      1   
00b3400e4ff64ee68ce9ada1d0c222f0 0b1e1539f2cc45b7b9fa7c272da2e1d7      1   
                                 3f207df678b143eea3cee63160fa8bed      1   
                                 5a8bc65990b245e5a138643cd4eb9837      1   
                                 ae264e3637204a6fb9bb56bc8210ddfd      1   

                                                                   value  \
person                           offer id                                  
00aee28bbb3848dd8a31f0c91dc267dd ae264e3637204a6fb9bb56bc8210ddfd      1   
                                 f19421c1d4aa40978ebb69ca19b0e20d      1   
00b18b535d6d4f779dea4dc9ac451478 2298d6c36e964ae4a3e7e9706d1fb8c2      2   
                                 5a8bc65990b245e5a138643cd4eb9837      1   
                                 ae264e3637204a6fb9bb56bc8210ddfd      1   
                                 fafdcd668e3743c1bb461111dcafc2a4      1   
00b3400e4ff64ee68ce9ada1d0c222f0 0b1e1539f2cc45b7b9fa7c272da2e1d7      1   
                                 3f207df678b143eea3cee63160fa8bed      1   
                                 5a8bc65990b245e5a138643cd4eb9837      1   
                                 ae264e3637204a6fb9bb56bc8210ddfd      1   

                                                                   time  \
person                           offer id                                 
00aee28bbb3848dd8a31f0c91dc267dd ae264e3637204a6fb9bb56bc8210ddfd     1   
                                 f19421c1d4aa40978ebb69ca19b0e20d     1   
00b18b535d6d4f779dea4dc9ac451478 2298d6c36e964ae4a3e7e9706d1fb8c2     2   
                                 5a8bc65990b245e5a138643cd4eb9837     1   
                                 ae264e3637204a6fb9bb56bc8210ddfd     1   
                                 fafdcd668e3743c1bb461111dcafc2a4     1   
00b3400e4ff64ee68ce9ada1d0c222f0 0b1e1539f2cc45b7b9fa7c272da2e1d7     1   
                                 3f207df678b143eea3cee63160fa8bed     1   
                                 5a8bc65990b245e5a138643cd4eb9837     1   
                                 ae264e3637204a6fb9bb56bc8210ddfd     1   

                                                                   amount  \
person                           offer id                                   
00aee28bbb3848dd8a31f0c91dc267dd ae264e3637204a6fb9bb56bc8210ddfd       0   
                                 f19421c1d4aa40978ebb69ca19b0e20d       0   
00b18b535d6d4f779dea4dc9ac451478 2298d6c36e964ae4a3e7e9706d1fb8c2       0   
                                 5a8bc65990b245e5a138643cd4eb9837       0   
                                 ae264e3637204a6fb9bb56bc8210ddfd       0   
                                 fafdcd668e3743c1bb461111dcafc2a4       0   
00b3400e4ff64ee68ce9ada1d0c222f0 0b1e1539f2cc45b7b9fa7c272da2e1d7       0   
                                 3f207df678b143eea3cee63160fa8bed       0   
                                 5a8bc65990b245e5a138643cd4eb9837       0   
                                 ae264e3637204a6fb9bb56bc8210ddfd       0   

                                                                   offer_id  \
person                           offer id                                     
00aee28bbb3848dd8a31f0c91dc267dd ae264e3637204a6fb9bb56bc8210ddfd         0   
                                 f19421c1d4aa40978ebb69ca19b0e20d         0   
00b18b535d6d4f779dea4dc9ac451478 2298d6c36e964ae4a3e7e9706d1fb8c2         0   
                    

Nem todos os usuários receberam todas as ofertas disponíveis.  

Será que os que receberam foi efetivo a conversão?  

Pontos a organizar no dataset:
1. Quais usuários receberam quais ofertas
2. Quantos leram aquela oferta
3. Quantos completaram a oferta
4. Quanto foi o income para quem recebeu cada oferta

O problema parece caminhar para usar variáveis dummies.   
Cada linha será um usuário, que recebeu cada uma das ofertas diferentes, gerou uma resposta diferente e gastou uma quantidade diferente. Cada usuário também tem seus próprio dados demográficos, que podem servir para clusterização.

In [12]:
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

Quando o usuário compra sem mesmo receber uma oferta, significa que ele não precisa de ofertas para comprar, o que o torna mais barato para as campanhas da empresa

O tempo vai determinar os status das ofertas. Assim, ele deve ser consolidado e resumido do dataset, usado somente para os cálculos

Campos a serem criados:
1. User
1. Ofertas recebidas
1. Ofertas visualizadas
1. Ofertas completadas
1. Transações totais

Portfolio tem 10 ids únicos  
Para cada uma dessas promoções, criar as colunas para cada usuários:   
enviada, vista, completada, transação, gasto no envio, quantidade enviada, quantidade vista, quantidade   

Quantidade de linhas  
17000 usuários, 10 * 5 colunas = 850 mil células  

É necessário analisar usuário por usuário, criando sua time line e definindo os campos  

Duas opções:
via iterrows() ou via merge  

testar qual é o mais rápido

In [13]:
cond = transcript.person == 1515
transcript.loc[cond]

,person,event,value,time


In [14]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


Passo a passo para tratamento dos dados:

1. Agrupar as ofertas recebidas

1. Verificar se essas ofertas foram visualizadas dentro do intervalo de validade  
   Contabilizar taxa de visualização

1. Verificar se essas ofertas foram completadas dentro do prazo de validade
   Calcular taxa de completas

1. Verificar quais transações foram realizadas após visualizações da oferta
   Somar total

1. Somar transações sem relações com as ofertas

# Transformando em uma função

In [15]:
# Functions

def get_subset(user_df, type, suffix, dict_keys):
    '''
    Get a dataset with different events and separates a subsets
    with specific type
    Input:
        user_df - (dataframe) - datafram with event of an user
        type - (string) - 'offer received', 'offer viewed', 'offer completed', 
        'transaction'
        suffix - (string) - a suffix to indtify the variables
        dict_keys - (list) - list of dict keys to extract from value column
    Output:
        df - (dataframe) - dataframe with subset
    '''
    df = user_df.loc[user_df.event==type]
    df = df[['time', 'value']]
    df = df.rename(columns={'time': 'time' + suffix})
    
    if type == 'offer received':
        df['time_next'] = df.shift(-1, 
            fill_value=transcript.time.max())['time' + suffix]
    
    # Extract value from dict
    for key in dict_keys:
        key_ = key.replace(' ', '_') # to replace empty space
        df[key_ + suffix] = df['value'].map(lambda d: d.get(key))
    df = df.drop(columns='value')

    return df 

def merge_and_filter(df_left, 
    df_right, on_left, on_right,
    col_filter, col_compare):
    '''
    Merge two dataframes and apply a filter in result
    Input:
        df_left - (dataframe) - left dataframe to merge
        df_right - (dataframe) - right dataframe to merge
        on_left - (string) - key to use in merge for left
        on_right - (string) - key to use in merge for right
        col_filter - (string) - column to apply some filters
        col_compare - (list of string) - list with columns to apply filters
    Output:
        df - (dataframe) - dataframe merged and filtered 
    '''

    df = df_left.merge(df_right, 
    left_on=on_left,
    right_on=on_right,
    how='left'
    )
    
    condition1 = (df[col_filter] >= df[col_compare[0]]) 
    condition2 = (df[col_filter] <= df[col_compare[1]])
    condition3 = (pd.isna(df[col_filter])) # keep not seen offers in datase
    
    df = df.loc[condition1 & condition2 | condition3]

    return df

def get_viewed(row):
    '''
    Auxiliar function to account for valid views
    Input: row, a dataframe coming from apply lambda
    Outpu: the validation of viewed offer
    '''
    if not pd.isna(row['time_vie']):
        if row['time_vie'] < row['period_max']:
            return 1 
        else:
            return 0
    else:
        return np.nan

def get_complete(row):
    '''
    Auxliar function to account the offer complete after viewd
    Input: row, a datafram row coming from apply lambda
    Outpu: complete offer after viewed
    '''
    if not pd.isna(row['time_com']):
        if row['time_com'] > row['time_vie']: 
            return 1 
        else:
            return 0
    else:
        return np.nan

In [16]:
def get_offer_table_user(user):
    '''
    Main function to extract informations about offers made for users.
    The ideia is analyse timeline of received, viewed and completed offer
    made by user and build a dataset with this consolitaded information.
    Input:
        user - the user id
    Output:
        offer_df - dataframe with consolitaded informations
    '''
    cond = transcript.person == user #1510
    user_df = transcript.loc[cond]

    # Resume do protifolio
    short_portifolio = portfolio[['id', 'duration']]

    # 1 - Creating the subsets
    # Received offers
    received_df = get_subset(user_df,'offer received', '_rec', ['offer id'] )
    # If user did not received any offer, skip it and return empty dataframe
    if received_df.size == 0:
        return pd.DataFrame()

    # Get info from portifolio
    received_df = received_df. \
        merge(short_portifolio, left_on='offer_id_rec', right_on='id'). \
        drop(columns='id')
    # Viewed offers
    viewed_df = get_subset(user_df, 'offer viewed', '_vie', ['offer id'] )
    # Completed offers
    completed_df = get_subset(user_df, 'offer completed', '_com', ['offer_id', 'reward'])
    # Transactions
    transaction_df = get_subset(user_df, 'transaction', '_tra', ['amount'])

    # 2 - Analizing the timeline
    # 2.1 - Visualized
    # Get the visualized offers
    offer_df = merge_and_filter(received_df, viewed_df, 
        'offer_id_rec', 'offer_id_vie',
        'time_vie', ['time_rec', 'time_next']
        )

    # Calculating the max time valid for offer
    offer_df['period_max'] = offer_df.time_rec + offer_df.duration*24

    offer_df['valid_view'] = offer_df.apply(lambda row: get_viewed(row), axis=1)

    # Transactions from user, influenced by offers
    offer_df['tra_offer_infl'] = 0

    # 2.2 - Transactions
    # Iterate over offers dataset and searching in the transactions the intervals
    # considered to be influenced by an offer

    for idx, _ in offer_df.iterrows():
        time_vie  = offer_df['time_vie'].at[idx]
        time_max  = offer_df['period_max'].at[idx]
        # time_next = offer_df['time_next'].at[idx]

        # Initialize variable
        sum_tra_infl = 0
        # Itarete over transactions
        for jdx, _ in transaction_df.iterrows():
            # Time of transactions
            time_tra = transaction_df['time_tra'].at[jdx]
            amo_tra  = transaction_df['amount_tra'].at[jdx]
        
            if (time_tra > time_vie and 
                time_tra <= time_max #and 
                # time_tra <= time_next
                ):
                sum_tra_infl += amo_tra
        
        # Assing to that offer
        offer_df['tra_offer_infl'].at[idx] = sum_tra_infl
    
    # 2.2 - Complete offers
    # Complete offers
    offer_df = merge_and_filter(offer_df, completed_df,
        'offer_id_rec', 'offer_id_com',
        'time_com', ['time_rec', 'period_max']
        )

    # The same offer can be sent to a user and be completed together, 
    # generating duplicates
    offer_df = offer_df.drop_duplicates(
        subset=['time_rec', 'offer_id_rec', 'time_vie']
        )

    offer_df['completed_after_view'] = offer_df.apply(lambda row: get_complete(row), axis=1)

    # 2.3 - Final
    # Getting the status for offers
    offer_df['viewed'] = offer_df.apply(lambda r: 
        1 if not pd.isna(r['offer_id_vie']) else 0,
        axis=1
        )

    offer_df['completed'] = offer_df.apply(lambda r: 
        1 if not pd.isna(r['offer_id_com']) else 0,
        axis=1
        )

    # Selecting just necessary columns
    offer_df = offer_df[['offer_id_rec', 'valid_view', 'tra_offer_infl',
       'reward_com', 'completed_after_view', 'viewed', 'completed']]

    return offer_df

def expand_portifolio(offer_df, portfolio_df=portfolio.copy()):
    '''
    Take the informations from portifolio dataframe and put it
    in the offer dataframe
    Input:
        offer_df - (dataframe) dataframe with offer by user treated
        protifolio_df - (dataframe) dataframe with features of protifolio
    Output
        offer_df - (dataframe) the original dataframe with expanded
        information about offers
    '''

    # columns_portifolio = ['offer_' + column for column in portfolio_df.columns]
    # portfolio_df.columns = columns_portifolio

    offer_df = offer_df.merge(portfolio_df,
        left_on='offer_id_rec',
        right_on='id'
        )
    
    offer_df = offer_df.drop(columns=['channels', 'id', 'duration'])

    offer_df = offer_df.rename(columns={
        'reward': 'offer_reward', 
        'difficulty': 'offer_difficulty'
        }
    )

    return offer_df

def group_offer_df(offer_df, map_dict=map_portifolio):
    '''
    After an offer dataframe be created for an user, this function
    group the data and get dummies for each offer present in dataset
    Input:
        offer_df - (dataframe) datafram with data about offer for a user
        map_dict - (dict) a dictonary create initialy to converte hex to int
    Output:
        offer_df - (dataframe) dataframe with dummies variables and grouped
        data

    About metrics
        viewed_rate - the rate of views for an offer
        completed_rate - the rate of completes for an offer
        tra_offer_infl - the total of transaction because of an offer
        valid_view - for viewed offers, the rate of vizualizations in 
        validy period of an offer
        completed_after_view_rate - for complete offer, the rate of that
        was complete after was visualize
        reward - for complete offer, the total of reward won by user
    '''
    # Check if the offer_df is not empty
    if offer_df.size == 0:
        return pd.DataFrame()
    
    # Group the data and create the metrics
    offer_df = offer_df.groupby('offer_id_rec', as_index=False).agg(
        viewed_rate=('viewed', 'mean'),
        completed_rate=('completed', 'mean'),
        tra_offer_infl=('tra_offer_infl', 'sum'),
        valid_view_rate=('valid_view', 'mean'),
        completed_after_view_rate=('completed_after_view', 'mean'),
        reward_won=('reward_com', 'sum')
        )

    offer_df = expand_portifolio(offer_df)

    offer_df['offer_id_rec'] = offer_df['offer_id_rec'].map(map_dict)

    # offer_df = pd.get_dummies(offer_df, columns=['offer_id_rec', 'type'])


    return offer_df


def get_total_transaction(user):
    '''
    Some transactions might not be related with offer received, but just
    a commom one. This function get the total of transactions made by
    an user, because of an offer or not
    Input:
        user - the user id
    Output:
        total_tra - (float) total transaction
    '''

    cond = transcript.person == user #1510
    user_df = transcript.loc[cond]

    # Transactions
    transaction_df = get_subset(user_df, 'transaction', '_tra', ['amount'])

    total_tra = transaction_df['amount_tra'].sum()

    return total_tra



In [21]:
# Create the complete dataframe

def create_user_offer_df(user):
    '''
    Function to get the user and apply previous functions
    to extract informations about offers
    Input:
        user - (str, int) user id
    Output:
        user_offer_df - (dataframe) dataframe with informations about user and
        offers
    '''
    user_offer_df = get_offer_table_user(user)
    user_offer_df = group_offer_df(user_offer_df)
    user_offer_df['user_id'] = user

    user_offer_df['user_id'] = user_offer_df['user_id'].map(map_profile)

    return user_offer_df


In [18]:
create_user_offer_df('8e0c1ffebf2a47d281210fabbd41d56a')

,offer_id_rec,viewed_rate,completed_rate,tra_offer_infl,valid_view_rate,completed_after_view_rate,reward_won,offer_reward,offer_difficulty,offer_type,user_id
0,4,1,1,170,1,1.0,5.0,5,20,discount,918
1,5,1,1,981,1,0.0,6.0,3,7,discount,918
2,7,1,0,18,1,NaN,0.0,0,0,informational,918
3,8,1,1,42,1,1.0,5.0,5,5,bogo,918


In [20]:
# Group final dataset for offer received

temp = get_offer_table_user('8e0c1ffebf2a47d281210fabbd41d56a')

temp

# a = expand_portifolio(temp)

# group_offer_df(temp)

# pd.get_dummies(a, columns=['offer_id_rec', 'offer_offer_type'])
# group_offer_df(temp)

,offer_id_rec,valid_view,tra_offer_infl,reward_com,completed_after_view,viewed,completed
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,170,5.0,1.0,1,1
1,f19421c1d4aa40978ebb69ca19b0e20d,1,42,5.0,1.0,1,1
2,5a8bc65990b245e5a138643cd4eb9837,1,9,NaN,NaN,1,0
3,5a8bc65990b245e5a138643cd4eb9837,1,9,NaN,NaN,1,0
4,2298d6c36e964ae4a3e7e9706d1fb8c2,1,494,3.0,0.0,1,1
7,2298d6c36e964ae4a3e7e9706d1fb8c2,1,487,3.0,0.0,1,1


In [41]:
# Iterate by user and get dataframes

users = transcript.person.unique()[:3400]
# np.random.shuffle(users)

dfs = []
cnt = 0
for user in users:
    cnt += 1
    try:
        df = create_user_offer_df(user)
        dfs.append(df)
    except:
        print(user)
    print((cnt/17000)*100, end="\r")


user_offer_df = pd.concat(dfs).reset_index(drop=True)

user_offer_df.to_csv('part1.csv')


In [40]:
user_offer_df

,offer_id_rec,viewed_rate,completed_rate,tra_offer_infl,valid_view_rate,completed_after_view_rate,reward_won,offer_reward,offer_difficulty,offer_type,user_id
0,4,1,0,0,1,NaN,0.0,5,20,discount,15
1,5,1,1,125,1,0.5,6.0,3,7,discount,15
2,6,1,1,69,1,1.0,2.0,2,10,discount,15
3,4,1,0,0,1,NaN,0.0,5,20,discount,17
4,5,1,0,0,1,NaN,0.0,3,7,discount,17
5,1,1,0,0,1,NaN,0.0,10,10,bogo,17
6,7,1,0,0,1,NaN,0.0,0,0,informational,17
7,0,1,0,0,1,NaN,0.0,10,10,bogo,17
8,4,1,0,3,1,NaN,0.0,5,20,discount,3746
9,9,1,0,2,1,NaN,0.0,2,10,discount,3746
